In [10]:
import osmnx as ox
import networkx as nx
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt
import numpy as np
import json
import hashlib 


In [11]:
# distance from station for which adjacent nodes should be retrieved
distance=100

In [12]:
def get_central_node(graph,lat, long,distance=100):
    """given a geocoded loction and a graph return 
        find the node closests to the location """
    graph_df,_ = ox.graph_to_gdfs(graph)
    graph_tree = KDTree(graph_df[['y', 'x']], metric='euclidean')
    adress_idx = graph_tree.query([(lat,long)], k=1, return_distance=False)[0]
    closest_node = graph_df.iloc[adress_idx].index.values[0]
    return closest_node

    

In [13]:
def open_json(file_path):
    with open(file_path) as file:
        data_dict = json.loads(file.read())
    return data_dict
    

In [14]:
def get_neighbourhood(graph):
    '''extract the lat long locations'''
    lats = [float(y) for _, y in graph.nodes(data='y')]
    lngs = [float(x) for _, x in graph.nodes(data='x')]
    return zip(lats, lngs)

In [15]:
def process_graph(graph, data_dict):
    """process a single graph, getting all adjacent nodes and 
    and  add them to a the dictionary mapping the stations"""
    
#     getting all information from the graph object
    zipper = get_neighbourhood(graph)
    nodes = list(graph.nodes)
    lat_n_longs = list(zipper)
    
#     mapping adjacent streets
    
    adj_list = list(graph.adjacency())
    for i, node in enumerate(nodes):
        data_dict['stations'][str(node)]={}
        data_dict['stations'][str(node)]['type'] = 'street'
        data_dict['stations'][str(node)]['lat'] = lat_n_longs[i][0]
        data_dict['stations'][str(node)]['lng'] = lat_n_longs[i][1]
        data_dict['stations'][str(node)]['next_stops'] = list(adj_list[i][1].keys())
    
    return

In [34]:
def process_stations(file_path, new_file,distance=100):
    """processing all stations from the input file"""
#     opening the json and converting to dict
    data_dict = open_json(file_path)
    temp_dict = {'stations':{}}
    for key in data_dict['stations'].keys():
#         creating the graph and finding the closes node to the station and appending it
        
        lat=data_dict['stations'][key]['lat']
        long=data_dict['stations'][key]['lng']
        graph=ox.core.graph_from_point((lat,long), distance=distance)
        central_node=get_central_node(graph, lat, long)
        data_dict['stations'][key]['next_stops'].append(central_node)
        
        process_graph(graph,temp_dict)    
        temp_dict['stations'][str(central_node)]['next_stops'].append(key)
        return temp_dict,central_node, graph
        
    data_dict['stations'].update(temp_dict['stations'])
    
    with open(new_file,'w') as file:
        data_dict_as_json=json.dump(data_dict,file,default=str)
            

In [76]:
def process_stations_debug(file_path, new_file,distance=100):
    """processing all stations from the input file"""
#     opening the json and converting to dict
    data_dict = open_json(file_path)
    temp_dict = {'stations':{}}
    for key in data_dict['stations'].keys():
#         creating the graph and finding the closes node to the station and appending it
        print(key)
        lat=data_dict['stations'][key]['lat']
        long=data_dict['stations'][key]['lng']
        graph=ox.core.graph_from_point((lat,long), distance=distance)
        central_node=get_central_node(graph, lat, long)
        data_dict['stations'][key]['next_stops'].append(central_node)
        
        process_graph(graph,temp_dict)    
        temp_dict['stations'][str(central_node)]['next_stops'].append(key)
        
        return temp_dict,central_node, graph
        
    data_dict['stations'].update(temp_dict['stations'])
    
    with open(new_file,'w') as file:
        data_dict_as_json=json.dump(data_dict,file,default=str)
            

In [77]:
data,node,g=process_stations_debug('data/test.json','data/test_out.json')

2094266776


In [36]:
data['stations'][str(node)]

{'type': 'street',
 'lat': 52.5216906,
 'lng': 13.4140824,
 'next_stops': [4934187009, 2777153528, 2777153497, '2094266776']}

In [125]:
hops=np.random.randint(3,10)
hop=0
path=[]
station=str(2094266776)
start=node
current_node=start
while hop <= hops: 
    print(current_node)
    next_node = np.random.choice(data['stations'][str(current_node)]['next_stops'])
    while next_node == station:
        if len(data['stations'][str(current_node)]['next_stops']):
            break
        else:
            np.random.choice(data['stations'][str(current_node)]['next_stops'])
    path.append(current_node)
    current_node = next_node
    hop = hop+1

3810179831
2777153497
2777153451
2777153445
2777153497
2777153508
2777153497
3810179831


In [120]:
path

[3810179831,
 '2777153528',
 4947074226,
 2777153459,
 2777153457,
 2777153459,
 2777153457]

In [62]:
hop

1